In [36]:
import pandas as pd
import numpy as np
import requests

In [37]:

'''
Get data from 30-06-2020 to 31-12-2021 to train machine learning model
API: https://www.energidataservice.dk/guides/api-guides
'''

base_url = "https://api.energidataservice.dk/dataset/ConsumptionDE35Hour"

params = {
    "start": "2020-06-30",
    "end": "2022-01-01",
    "columns": "HourDK,PriceArea,ConsumerType_DE35,TotalCon",
    "limit": "0"
}

response = requests.get(base_url, params=params)
data = response.json()['records']

df = pd.DataFrame(data)

KeyboardInterrupt: 

In [ ]:
df.head()

,HourDK,PriceArea,ConsumerType_DE35,TotalCon
0,2021-12-31 23:00:00,DK1,111,77141
1,2021-12-31 23:00:00,DK1,112,2974
2,2021-12-31 23:00:00,DK1,119,25668
3,2021-12-31 23:00:00,DK1,121,346616
4,2021-12-31 23:00:00,DK1,122,66095


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003276 entries, 0 to 1003275
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   HourDK             1003276 non-null  datetime64[ns]
 1   PriceArea          1003276 non-null  object        
 2   ConsumerType_DE35  1003276 non-null  object        
 3   TotalCon           1003276 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 30.6+ MB


In [ ]:
# Change hour column to datetime dtype
df['HourDK'] = pd.to_datetime(df['HourDK'], errors = 'coerce')

# change consumer type to string
df['ConsumerType_DE35'] = df['ConsumerType_DE35'].astype(str)

In [ ]:
# Feature Engineering
df['HourDK_year'] = df['HourDK'].dt.year.astype(np.int64)
df['HourDK_month'] = df['HourDK'].dt.month.astype(np.int64)
df['HourDK_day'] = df['HourDK'].dt.day.astype(np.int64)
df['HourDK_hour'] = df['HourDK'].dt.hour.astype(np.int64)
df['HourDK_weekofyear'] = df['HourDK'].dt.isocalendar().week.astype(np.int64)
df['HourDK_dayofweek'] = df['HourDK'].dt.dayofweek.astype(np.int64)

df= df.drop(columns=['HourDK'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003276 entries, 0 to 1003275
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   PriceArea          1003276 non-null  object
 1   ConsumerType_DE35  1003276 non-null  object
 2   TotalCon           1003276 non-null  int64 
 3   HourDK_year        1003276 non-null  int64 
 4   HourDK_month       1003276 non-null  int64 
 5   HourDK_day         1003276 non-null  int64 
 6   HourDK_hour        1003276 non-null  int64 
 7   HourDK_weekofyear  1003276 non-null  int64 
 8   HourDK_dayofweek   1003276 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 68.9+ MB


In [ ]:
from sklearn.model_selection import train_test_split

# Define features and target variable
X = df.drop(columns=['TotalCon'])
y = df['TotalCon']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
X_train.loc[1001349]

PriceArea             DK2
ConsumerType_DE35     310
HourDK_year          2020
HourDK_month            7
HourDK_day              1
HourDK_hour             1
HourDK_weekofyear      27
HourDK_dayofweek        2
Name: 1001349, dtype: object

In [ ]:
# modelling 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [ ]:
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown='ignore')

transformed_X_train = X_train.copy()
transformed_y_train = y_train

numeric_features = ['HourDK_year','HourDK_month','HourDK_day','HourDK_hour','HourDK_weekofyear','HourDK_dayofweek']
categorical_features = ['PriceArea', 'ConsumerType_DE35']

transformed_X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])

categorical_encoded = encoder.fit_transform(X_train[categorical_features]).toarray()
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(categorical_features))
transformed_X_train.drop(categorical_features, axis=1, inplace=True)
transformed_X_train = pd.concat([transformed_X_train.reset_index(drop=True), categorical_encoded_df], axis=1)


In [ ]:
datetime_records = transformed_X_train[transformed_X_train.apply(lambda row: any(isinstance(val, pd.Timestamp) for val in row), axis=1)]
print(datetime_records)

Empty DataFrame
Columns: [HourDK_year, HourDK_month, HourDK_day, HourDK_hour, HourDK_weekofyear, HourDK_dayofweek, PriceArea_DK1, PriceArea_DK2, ConsumerType_DE35_111, ConsumerType_DE35_112, ConsumerType_DE35_119, ConsumerType_DE35_121, ConsumerType_DE35_122, ConsumerType_DE35_123, ConsumerType_DE35_130, ConsumerType_DE35_211, ConsumerType_DE35_212, ConsumerType_DE35_215, ConsumerType_DE35_220, ConsumerType_DE35_310, ConsumerType_DE35_320, ConsumerType_DE35_330, ConsumerType_DE35_340, ConsumerType_DE35_350, ConsumerType_DE35_360, ConsumerType_DE35_370, ConsumerType_DE35_381, ConsumerType_DE35_382, ConsumerType_DE35_390, ConsumerType_DE35_410, ConsumerType_DE35_421, ConsumerType_DE35_422, ConsumerType_DE35_431, ConsumerType_DE35_432, ConsumerType_DE35_433, ConsumerType_DE35_441, ConsumerType_DE35_442, ConsumerType_DE35_443, ConsumerType_DE35_444, ConsumerType_DE35_445, ConsumerType_DE35_446, ConsumerType_DE35_447, ConsumerType_DE35_450, ConsumerType_DE35_461, ConsumerType_DE35_462, Cons

In [ ]:
transformed_X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802620 entries, 0 to 802619
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   HourDK_year            802620 non-null  float64
 1   HourDK_month           802620 non-null  float64
 2   HourDK_day             802620 non-null  float64
 3   HourDK_hour            802620 non-null  float64
 4   HourDK_weekofyear      802620 non-null  float64
 5   HourDK_dayofweek       802620 non-null  float64
 6   PriceArea_DK1          802620 non-null  float64
 7   PriceArea_DK2          802620 non-null  float64
 8   ConsumerType_DE35_111  802620 non-null  float64
 9   ConsumerType_DE35_112  802620 non-null  float64
 10  ConsumerType_DE35_119  802620 non-null  float64
 11  ConsumerType_DE35_121  802620 non-null  float64
 12  ConsumerType_DE35_122  802620 non-null  float64
 13  ConsumerType_DE35_123  802620 non-null  float64
 14  ConsumerType_DE35_130  802620 non-nu

In [ ]:
transformed_X_test = X_test.copy()
transformed_y_test = y_test

numeric_features = ['HourDK_year','HourDK_month','HourDK_day','HourDK_hour','HourDK_weekofyear','HourDK_dayofweek']
categorical_features = ['PriceArea', 'ConsumerType_DE35']

transformed_X_test[numeric_features] = scaler.transform(X_test[numeric_features])

categorical_encoded_test = encoder.transform(X_test[categorical_features]).toarray()
categorical_encoded_test_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(categorical_features))
transformed_X_test.drop(categorical_features, axis=1, inplace=True)
transformed_X_test = pd.concat([transformed_X_test.reset_index(drop=True), categorical_encoded_df], axis=1)

In [ ]:
transformed_X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802620 entries, 0 to 802619
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   HourDK_year            200656 non-null  float64
 1   HourDK_month           200656 non-null  float64
 2   HourDK_day             200656 non-null  float64
 3   HourDK_hour            200656 non-null  float64
 4   HourDK_weekofyear      200656 non-null  float64
 5   HourDK_dayofweek       200656 non-null  float64
 6   PriceArea_DK1          802620 non-null  float64
 7   PriceArea_DK2          802620 non-null  float64
 8   ConsumerType_DE35_111  802620 non-null  float64
 9   ConsumerType_DE35_112  802620 non-null  float64
 10  ConsumerType_DE35_119  802620 non-null  float64
 11  ConsumerType_DE35_121  802620 non-null  float64
 12  ConsumerType_DE35_122  802620 non-null  float64
 13  ConsumerType_DE35_123  802620 non-null  float64
 14  ConsumerType_DE35_130  802620 non-nu

In [ ]:
rfr = RandomForestRegressor(n_estimators=200, max_depth=20)
rfr.fit(transformed_X_train, transformed_y_train)

KeyboardInterrupt: 

In [ ]:
prediction = rfr.predict(transformed_X_test)
mse = mean_squared_error(prediction, transformed_y_test)

In [ ]:
# prepprcoessing pipeline
numeric_features = ['HourDK_year','HourDK_month','HourDK_day','HourDK_hour','HourDK_weekofyear','HourDK_dayofweek']
categorical_features = ['PriceArea', 'ConsumerType_DE35']

preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_features),
        ('ohe', OneHotEncoder(), categorical_features)],
    remainder = 'passthrough')

# define different candidate models in a dictionary
models_params = {
    'random_forest': {
        'model': RandomForestRegressor(),
        'params':{
            'randomforestregressor__n_estimators': [100,200,300],
            'randomforestregressor__max_depth': [None, 10, 20, 30]
        }
    },
    'gradient_boost': {
        'model': GradientBoostingRegressor(),
        'params':{
            'gradientboostingregressor__n_estimators': [100, 200, 300],
            'gradientboostingregressor__max_depth': [3, 4, 5],
            'gradientboostingregressor__learning_rate': [0.01, 0.1, 0.2]
        }
    },
    'svr': {
        'model': SVR(gamma='auto'),
        'params':{
            'svr__kernel': ['rbf', 'poly', 'sigmoid'],
            'svr__C': [0.1, 1, 10, 100, 1000]
        }
    }
}

In [ ]:
# run GridSearchCV to find the best model
scores = []
best_estimators = {}

for algo, mp in models_params.items():
    pipe = make_pipeline(preprocessor, mp['model'])
    rgs = GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False)
    rgs.fit(X_train, y_train)

    # best scores of each model
    scores.append({
        'model': algo,
        'best_score': rgs.best_score_,
        'best_params': rgs.best_params_
    })
    best_estimators[algo] = rgs.best_estimator_

df=pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df